# Pre-Processing and Training Data

# Imports

In [45]:
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import __version__ as sklearn_version
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV, learning_curve
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime


# Load Data

In [46]:
df_merged = pd.read_csv('/Users/swatisharma/Documents/GitHub/Capstone2_Demand_Forecast/df_merged_features.csv')
df_merged.head().T

,0,1,2,3,4
record_ID,1,2,3,4,5
week,2011-01-17,2011-01-17,2011-01-17,2011-01-17,2011-01-17
store_id,8091,8091,8091,8091,8091
sku_id,216418,216419,216425,216233,217390
total_price,99.0375,99.0375,133.95,133.95,141.075
base_price,111.8625,99.0375,133.95,133.95,141.075
is_featured_sku,0,0,0,0,0
is_display_sku,0,0,0,0,0
units_sold,20,28,19,44,52
year,2011,2011,2011,2011,2011


In [47]:
#Drop the column "week"
df_merged = df_merged.drop('week', axis=1)


In [48]:
df_merged.shape

(150150, 13)

In [49]:
df_merged["month"].unique()

array([ 1,  7,  2,  3,  4, 11,  9,  5,  6,  8, 12, 10])

In [50]:
#Create dummy or indicator features for categorical variable "month"
result = pd.get_dummies(df_merged, columns = ["month"], prefix = "M", dtype=int)
result

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,year,day,...,M_3,M_4,M_5,M_6,M_7,M_8,M_9,M_10,M_11,M_12
0,1,8091,216418,99.0375,111.8625,0,0,20,2011,17,...,0,0,0,0,0,0,0,0,0,0
1,2,8091,216419,99.0375,99.0375,0,0,28,2011,17,...,0,0,0,0,0,0,0,0,0,0
2,3,8091,216425,133.9500,133.9500,0,0,19,2011,17,...,0,0,0,0,0,0,0,0,0,0
3,4,8091,216233,133.9500,133.9500,0,0,44,2011,17,...,0,0,0,0,0,0,0,0,0,0
4,5,8091,217390,141.0750,141.0750,0,0,52,2011,17,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150145,212638,9984,223245,235.8375,235.8375,0,0,38,2013,7,...,0,0,0,0,0,0,1,0,0,0
150146,212639,9984,223153,235.8375,235.8375,0,0,30,2013,7,...,0,0,0,0,0,0,1,0,0,0
150147,212642,9984,245338,357.6750,483.7875,1,1,31,2013,7,...,0,0,0,0,0,0,1,0,0,0
150148,212643,9984,547934,141.7875,191.6625,0,1,12,2013,7,...,0,0,0,0,0,0,1,0,0,0


In [51]:
#Check if there are any nan values
result.isnull().sum(axis = 0)

record_ID          0
store_id           0
sku_id             0
total_price        1
base_price         0
is_featured_sku    0
is_display_sku     0
units_sold         0
year               0
day                0
average_price      0
revenue            0
M_1                0
M_2                0
M_3                0
M_4                0
M_5                0
M_6                0
M_7                0
M_8                0
M_9                0
M_10               0
M_11               0
M_12               0
dtype: int64

In [52]:
#Drop the column "total_price"
result = result.dropna(subset = ["total_price"])
result

,record_ID,store_id,sku_id,total_price,base_price,is_featured_sku,is_display_sku,units_sold,year,day,...,M_3,M_4,M_5,M_6,M_7,M_8,M_9,M_10,M_11,M_12
0,1,8091,216418,99.0375,111.8625,0,0,20,2011,17,...,0,0,0,0,0,0,0,0,0,0
1,2,8091,216419,99.0375,99.0375,0,0,28,2011,17,...,0,0,0,0,0,0,0,0,0,0
2,3,8091,216425,133.9500,133.9500,0,0,19,2011,17,...,0,0,0,0,0,0,0,0,0,0
3,4,8091,216233,133.9500,133.9500,0,0,44,2011,17,...,0,0,0,0,0,0,0,0,0,0
4,5,8091,217390,141.0750,141.0750,0,0,52,2011,17,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150145,212638,9984,223245,235.8375,235.8375,0,0,38,2013,7,...,0,0,0,0,0,0,1,0,0,0
150146,212639,9984,223153,235.8375,235.8375,0,0,30,2013,7,...,0,0,0,0,0,0,1,0,0,0
150147,212642,9984,245338,357.6750,483.7875,1,1,31,2013,7,...,0,0,0,0,0,0,1,0,0,0
150148,212643,9984,547934,141.7875,191.6625,0,1,12,2013,7,...,0,0,0,0,0,0,1,0,0,0


In [53]:
#Check if there are still nan values
result.isnull().sum(axis = 0)

record_ID          0
store_id           0
sku_id             0
total_price        0
base_price         0
is_featured_sku    0
is_display_sku     0
units_sold         0
year               0
day                0
average_price      0
revenue            0
M_1                0
M_2                0
M_3                0
M_4                0
M_5                0
M_6                0
M_7                0
M_8                0
M_9                0
M_10               0
M_11               0
M_12               0
dtype: int64

# Train/Test Split

Partition sizes with a 70/30 train/test split

In [54]:
len(result) * .7, len(result) * .3

(105104.29999999999, 45044.7)

In [55]:
X_train, X_test, y_train, y_test = train_test_split(result.drop(columns='units_sold'), 
                                                    result.units_sold, test_size=0.3, 
                                                    random_state=47)

In [56]:
X_train.shape, X_test.shape

((105104, 23), (45045, 23))

In [57]:
y_train.shape, y_test.shape

((105104,), (45045,))

In [58]:
#Save the ''record_ID', 'total_price', 'revenue' columns from the train/test data into units_sold_train and units_sold_test
#Then drop those columns from `X_train` and `X_test`. Use 'inplace=True'
column_list = ['record_ID', 'total_price', 'revenue']
column_list_train = X_train[column_list]
column_list_test = X_test[column_list]
X_train.drop(columns=column_list, inplace=True)
X_test.drop(columns=column_list, inplace=True)
X_train.shape, X_test.shape

((105104, 20), (45045, 20))

In [59]:
#Check the `dtypes` attribute of `X_train` to verify all features are numeric
X_train.dtypes

store_id             int64
sku_id               int64
base_price         float64
is_featured_sku      int64
is_display_sku       int64
year                 int64
day                  int64
average_price        int64
M_1                  int64
M_2                  int64
M_3                  int64
M_4                  int64
M_5                  int64
M_6                  int64
M_7                  int64
M_8                  int64
M_9                  int64
M_10                 int64
M_11                 int64
M_12                 int64
dtype: object

In [60]:
#Repeat this check for the test split in `X_test`
X_test.dtypes

store_id             int64
sku_id               int64
base_price         float64
is_featured_sku      int64
is_display_sku       int64
year                 int64
day                  int64
average_price        int64
M_1                  int64
M_2                  int64
M_3                  int64
M_4                  int64
M_5                  int64
M_6                  int64
M_7                  int64
M_8                  int64
M_9                  int64
M_10                 int64
M_11                 int64
M_12                 int64
dtype: object

We have only numeric features in our X now!

# Initial Not-Even-A-Model

In [61]:
#Calculate the mean of `y_train`
train_mean = y_train.mean()
train_mean

51.70557733292738

Using sklearn's DummyRegressor on the training data

In [62]:
#Fit the dummy regressor on the training data
#Hint, call its `.fit()` method with `X_train` and `y_train` as arguments
#Then print the object's `constant_` attribute and verify it's the same as the mean above
dumb_reg = DummyRegressor(strategy='mean')
dumb_reg.fit(X_train, y_train)
dumb_reg.constant_

array([[51.70557733]])

# Metrics

In [63]:
#Calculate the R^2 as defined above
def r_squared(y, ypred):
    """R-squared score.
    
    Calculate the R-squared, or coefficient of determination, of the input.
    
    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    ybar = np.sum(y) / len(y) #yes, we could use np.mean(y)
    sum_sq_tot = np.sum((y - ybar)**2) #total sum of squares error
    sum_sq_res = np.sum((y - ypred)**2) #residual sum of squares error
    R2 = 1.0 - sum_sq_res / sum_sq_tot
    return R2

In [64]:
y_tr_pred_ = train_mean * np.ones(len(y_train))
y_tr_pred_[:5]

array([51.70557733, 51.70557733, 51.70557733, 51.70557733, 51.70557733])

In [65]:
y_tr_pred = dumb_reg.predict(X_train)
y_tr_pred[:5]

array([51.70557733, 51.70557733, 51.70557733, 51.70557733, 51.70557733])

In [66]:
r_squared(y_train, y_tr_pred)

0.0

Exactly as expected, if we use the average value as your prediction, we get an 
 of zero on our training set. 

Make our predictions by creating an array of length the size of the test set with the single value of the (training) mean.

In [67]:
y_te_pred = train_mean * np.ones(len(y_test))
r_squared(y_test, y_te_pred)

-3.0974410798467744e-06

# Mean Absolute Error

This is very simply the average of the absolute errors:

 
 


In [68]:
#Calculate the MAE as defined above
def mae(y, ypred):
    """Mean absolute error.
    
    Calculate the mean absolute error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    abs_error = np.abs(y - ypred)
    mae = np.mean(abs_error)
    return mae

In [69]:
mae(y_train, y_tr_pred)

35.373134508889166

In [70]:
mae(y_test, y_te_pred)

35.23044624963676

# Mean Squared Error

Using another common metric for optimizing machine learning models is the mean squared error, which is simply the average of the square of the errors.

In [71]:
#Calculate the MSE as defined above
def mse(y, ypred):
    """Mean square error.
    
    Calculate the mean square error of the arguments

    Arguments:
    y -- the observed values
    ypred -- the predicted values
    """
    sq_error = (y - ypred)**2
    mse = np.mean(sq_error)
    return mse


In [72]:
mse(y_train, y_tr_pred)

3697.8953806759732

In [73]:
mse(y_test, y_te_pred)

3454.830889700147

In [74]:
np.sqrt([mse(y_train, y_tr_pred), mse(y_test, y_te_pred)])

array([60.81032298, 58.7778095 ])

# Initial Models

Impute missing values with median

In [98]:
#These are the values we'll use to fill in any missing values
X_defaults_median = X_train.median()
X_defaults_median

store_id             9371.0000
sku_id             222087.0000
base_price            205.9125
is_featured_sku         0.0000
is_display_sku          0.0000
year                 2012.0000
day                    16.0000
average_price         219.0000
M_1                     0.0000
M_2                     0.0000
M_3                     0.0000
M_4                     0.0000
M_5                     0.0000
M_6                     0.0000
M_7                     0.0000
M_8                     0.0000
M_9                     0.0000
M_10                    0.0000
M_11                    0.0000
M_12                    0.0000
dtype: float64

Apply the imputation to both train and test splits

In [99]:
#Call `X_train` and `X_test`'s `fillna()` method, passing `X_defaults_median` as the values to use
#Assign the results to `X_tr` and `X_te`, respectively
X_tr = X_train.fillna(X_defaults_median)
X_te = X_test.fillna(X_defaults_median)

# Scale the data

As we have features measured in many different units, with numbers that vary by orders of magnitude, we are starting off by scaling them to put them all on a consistent scale. The StandardScaler scales each feature to zero mean and unit variance.

In [100]:
#Call the StandardScaler`s fit method on `X_tr` to fit the scaler
#then use it's `transform()` method to apply the scaling to both the train and test split
#data (`X_tr` and `X_te`), naming the results `X_tr_scaled` and `X_te_scaled`, respectively
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

Train the model on the train split

In [101]:
lm = LinearRegression().fit(X_tr_scaled, y_train)

Make predictions using the model on both train and test splits

In [102]:
#Call the `predict()` method of the model (`lm`) on both the (scaled) train and test data
#Assign the predictions to `y_tr_pred` and `y_te_pred`, respectively
y_tr_pred = lm.predict(X_tr_scaled)
y_te_pred = lm.predict(X_te_scaled)

Assess model performance

In [103]:
#Calculate r^2 - train, test
median_r2 = r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)
median_r2

(0.2418874644975766, 0.24984706075882201)

In [114]:
#Now calculate the mean absolute error scores
# as we did above for R^2
# MAE - train, test
median_mae = mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)
median_mae

(30.363884392309107, 30.173017747506098)

In [115]:
# MSE - train, test
median_mse = mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)
median_mse

(2803.420843066961, 2591.643519026679)

# Impute missing values with the mean

In [106]:
#Calculate mean values for imputing missing values
# These are the values we'll use to fill in any missing values
X_defaults_mean = X_train.mean()
X_defaults_mean

store_id             9199.419128
sku_id             254910.859625
base_price            219.184582
is_featured_sku         0.095705
is_display_sku          0.132811
year                 2011.831662
day                    15.685607
average_price         219.000000
M_1                     0.092271
M_2                     0.099673
M_3                     0.084716
M_4                     0.092575
M_5                     0.107636
M_6                     0.084573
M_7                     0.077105
M_8                     0.068827
M_9                     0.069721
M_10                    0.068665
M_11                    0.084450
M_12                    0.069788
dtype: float64

Apply the imputation to both train and test splits

In [107]:
X_tr = X_train.fillna(X_defaults_mean)
X_te = X_test.fillna(X_defaults_mean)

Scale the data

In [108]:
scaler = StandardScaler()
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)
X_te_scaled = scaler.transform(X_te)

Train the model on the train split

In [109]:
lm = LinearRegression().fit(X_tr_scaled, y_train)

Make predictions using the model on both train and test splits

In [110]:
y_tr_pred = lm.predict(X_tr_scaled)
y_te_pred = lm.predict(X_te_scaled)

Assess model performance

In [111]:
r2_score(y_train, y_tr_pred), r2_score(y_test, y_te_pred)

(0.2418874644975766, 0.24984706075882201)

In [112]:
mean_absolute_error(y_train, y_tr_pred), mean_absolute_error(y_test, y_te_pred)

(30.363884392309107, 30.173017747506098)

In [113]:
mean_squared_error(y_train, y_tr_pred), mean_squared_error(y_test, y_te_pred)

(2803.420843066961, 2591.643519026679)

In next step we will perform modeling and build two to three different models including linear regression, randome forest etc., identify the best one and perform hyperparameter tuning. Also, access the data quality using various metrics. 